In [32]:
# !pip3 install scikit-image

In [2]:
import kagglehub
path = kagglehub.dataset_download("ankit1743/skyview-an-aerial-landscape-dataset") + "/Aerial_Landscapes"
path

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/Users/bytedance/.cache/kagglehub/datasets/ankit1743/skyview-an-aerial-landscape-dataset/versions/1/Aerial_Landscapes'

# machine learning

In [34]:
# base models
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', random_state=42) 
knn_model = KNeighborsClassifier(n_neighbors=5) 
forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

## color

In [10]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def extract_color_histogram(image, bins=(16, 16, 16)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
                        [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

dataset_path = path  

color_features = []
labels = []

for label in os.listdir(dataset_path):
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue  
    
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        try:
            image = cv2.imread(img_path)
            if image is None:
                continue
            image = cv2.resize(image, (100, 100))  
            hist = extract_color_histogram(image)
            color_features.append(hist)
            labels.append(label)
        except Exception as e:
            print(f"error here {img_path}: {e}")

x_train, x_test, y_train, y_test = train_test_split(
    color_features, labels, test_size=0.2, random_state=42)


knn model

In [ ]:

knn_model.fit(x_train, y_train)
knn_model.fit(x_train, y_train)

print("knn accuracy:", knn_model.score(x_test, y_test))
print(classification_report(y_test, knn_model.predict(x_test)))

knn accuracy: 0.5933333333333334
              precision    recall  f1-score   support

 Agriculture       0.44      0.48      0.46       168
     Airport       0.38      0.37      0.38       157
       Beach       0.62      0.40      0.49       156
        City       0.63      0.74      0.68       170
      Desert       0.74      0.94      0.83       170
      Forest       0.64      0.91      0.75       161
   Grassland       0.64      0.85      0.73       152
     Highway       0.58      0.24      0.34       173
        Lake       0.60      0.28      0.38       165
    Mountain       0.52      0.66      0.59       155
     Parking       0.65      0.71      0.67       157
        Port       0.61      0.56      0.59       149
     Railway       0.55      0.64      0.59       163
 Residential       0.67      0.80      0.73       148
       River       0.56      0.34      0.42       156

    accuracy                           0.59      2400
   macro avg       0.59      0.60      0.57    

svm

In [ ]:


svm_model.fit(x_train, y_train)

print("svm accuracy:", svm_model.score(x_test, y_test))
print(classification_report(y_test, svm_model.predict(x_test)))

svm accuracy: 0.685
              precision    recall  f1-score   support

 Agriculture       0.58      0.54      0.56       168
     Airport       0.47      0.54      0.50       157
       Beach       0.66      0.71      0.69       156
        City       0.80      0.75      0.78       170
      Desert       0.85      0.95      0.90       170
      Forest       0.83      0.88      0.85       161
   Grassland       0.72      0.89      0.80       152
     Highway       0.66      0.44      0.53       173
        Lake       0.66      0.48      0.56       165
    Mountain       0.67      0.62      0.64       155
     Parking       0.72      0.75      0.73       157
        Port       0.63      0.74      0.68       149
     Railway       0.62      0.72      0.67       163
 Residential       0.83      0.79      0.81       148
       River       0.57      0.50      0.53       156

    accuracy                           0.69      2400
   macro avg       0.68      0.69      0.68      2400
weight

forest

In [ ]:
forest_model.fit(x_train, y_train)

print("forest accuracy:", forest_model.score(x_test, y_test))
print(classification_report(y_test, forest_model.predict(x_test)))

forest accuracy: 0.7191666666666666
              precision    recall  f1-score   support

 Agriculture       0.66      0.69      0.67       168
     Airport       0.54      0.59      0.56       157
       Beach       0.72      0.80      0.76       156
        City       0.71      0.74      0.72       170
      Desert       0.92      0.96      0.94       170
      Forest       0.81      0.90      0.85       161
   Grassland       0.87      0.93      0.90       152
     Highway       0.68      0.45      0.54       173
        Lake       0.64      0.52      0.57       165
    Mountain       0.66      0.65      0.65       155
     Parking       0.83      0.80      0.82       157
        Port       0.81      0.77      0.79       149
     Railway       0.61      0.65      0.63       163
 Residential       0.74      0.86      0.80       148
       River       0.56      0.52      0.54       156

    accuracy                           0.72      2400
   macro avg       0.72      0.72      0.72 

## texture

### lbp

In [15]:
from skimage import feature

def extract_lbp_features(image, num_points=24, radius=8, eps=1e-7):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = feature.local_binary_pattern(gray, num_points, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
                             bins=np.arange(0, num_points + 3),
                             range=(0, num_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + eps) 
    return hist

texture_features = []
labels = []

for label in os.listdir(dataset_path):
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue  

    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        try:
            image = cv2.imread(img_path)
            if image is None:
                continue
            image = cv2.resize(image, (100, 100))
            lbp_hist = extract_lbp_features(image)
            texture_features.append(lbp_hist)
            labels.append(label)
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

x_train, x_test, y_train, y_test = train_test_split(
    texture_features, labels, test_size=0.2, random_state=42)

knn_model.fit(x_train, y_train)
print("LBP - KNN Accuracy:", knn_model.score(x_test, y_test))

svm_model.fit(x_train, y_train)
print("LBP - SVM Accuracy:", svm_model.score(x_test, y_test))

forest_model.fit(x_train, y_train)
print("LBP - RF Accuracy:", forest_model.score(x_test, y_test))


LBP - KNN Accuracy: 0.3229166666666667
LBP - SVM Accuracy: 0.20208333333333334
LBP - RF Accuracy: 0.40125


### sift

In [51]:
import cv2
import numpy as np

sift = cv2.SIFT_create()

# Store all descriptors and labels
sift_descriptors_all = []
sift_labels = []
sift_desc_per_image = []

for label in os.listdir(dataset_path):
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue

    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        try:
            image = cv2.imread(img_path)
            if image is None:
                continue

            image = cv2.resize(image, (100, 100))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            keypoints, descriptors = sift.detectAndCompute(gray, None)

            if descriptors is not None:
                sift_descriptors_all.extend(descriptors)
                sift_desc_per_image.append(descriptors)
                sift_labels.append(label)
            else:
                sift_desc_per_image.append(np.zeros((1, 128), dtype=np.float32))
                sift_labels.append(label)

        except Exception as e:
            print(f"Error processing {img_path}: {e}")


from sklearn.cluster import KMeans

n_clusters = 100
kmeans = KMeans(n_clusters=n_clusters, random_state=42, verbose=1)
kmeans.fit(np.asarray(sift_descriptors_all, dtype=np.float32))

def build_bovw_histograms(descriptors_list, kmeans, n_clusters):
    histograms = []

    for desc in descriptors_list:
        if desc is None or desc.size == 0:
            hist = np.zeros(n_clusters)
        else:
            labels = kmeans.predict(desc)
            hist, _ = np.histogram(labels, bins=np.arange(n_clusters + 1))
        histograms.append(hist.astype(np.float32))

    return np.array(histograms)


Initialization complete
Iteration 0, inertia 125249921024.0.
Iteration 1, inertia 86038921216.0.
Iteration 2, inertia 84445364224.0.
Iteration 3, inertia 83756949504.0.
Iteration 4, inertia 83373572096.0.
Iteration 5, inertia 83134087168.0.
Iteration 6, inertia 82968641536.0.
Iteration 7, inertia 82841075712.0.
Iteration 8, inertia 82733580288.0.
Iteration 9, inertia 82637799424.0.
Iteration 10, inertia 82551259136.0.
Iteration 11, inertia 82475098112.0.
Iteration 12, inertia 82410291200.0.
Iteration 13, inertia 82354962432.0.
Iteration 14, inertia 82307358720.0.
Iteration 15, inertia 82266513408.0.
Iteration 16, inertia 82230353920.0.
Iteration 17, inertia 82198306816.0.
Iteration 18, inertia 82169921536.0.
Iteration 19, inertia 82145026048.0.
Iteration 20, inertia 82123407360.0.
Iteration 21, inertia 82103861248.0.
Iteration 22, inertia 82086789120.0.
Iteration 23, inertia 82072010752.0.
Iteration 24, inertia 82058952704.0.
Iteration 25, inertia 82047401984.0.
Iteration 26, inertia 8

In [ ]:
sift_features = build_bovw_histograms(sift_desc_per_image, kmeans, n_clusters)
x_train, x_test, y_train, y_test = train_test_split(
    sift_features, sift_labels, test_size=0.2, random_state=42)

knn_model.fit(x_train, y_train)
print("SIFT - KNN Accuracy:", knn_model.score(x_test, y_test))

svm_model.fit(x_train, y_train)
print("SIFT - SVM Accuracy:", svm_model.score(x_test, y_test))

forest_model.fit(x_train, y_train)
print("SIFT - RF Accuracy:", forest_model.score(x_test, y_test))


SIFT - KNN Accuracy: 0.3433333333333333
SIFT - SVM Accuracy: 0.5141666666666667
SIFT - RF Accuracy: 0.43375


In [54]:
from sklearn.metrics import classification_report

y_pred = svm_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

 Agriculture       0.48      0.42      0.45       168
     Airport       0.46      0.36      0.40       157
       Beach       0.47      0.60      0.53       156
        City       0.63      0.62      0.63       170
      Desert       0.43      0.21      0.28       170
      Forest       0.61      0.53      0.57       161
   Grassland       0.36      0.81      0.50       152
     Highway       0.59      0.48      0.53       173
        Lake       0.47      0.41      0.43       165
    Mountain       0.60      0.57      0.58       155
     Parking       0.69      0.64      0.67       157
        Port       0.64      0.57      0.60       149
     Railway       0.46      0.44      0.45       163
 Residential       0.56      0.73      0.63       148
       River       0.43      0.38      0.40       156

    accuracy                           0.51      2400
   macro avg       0.53      0.52      0.51      2400
weighted avg       0.53   

# merging

since sift was more useful than lbp, we'll use just that in this case

In [55]:
combined_features = []
combined_labels = []

for label in os.listdir(dataset_path):
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue

    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        try:
            image = cv2.imread(img_path)
            if image is None:
                continue
            image = cv2.resize(image, (100, 100))

            color_feat = extract_color_histogram(image)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            kp, desc = sift.detectAndCompute(gray, None)

            if desc is None or desc.size == 0:
                desc = np.zeros((1, 128), dtype=np.float32)

            if desc.ndim == 1:
                desc = desc.reshape(1, -1)

            hist = build_histogram(desc, kmeans, n_clusters)

            # Combine features (early fusion)
            combined_feat = np.concatenate([color_feat, hist])
            combined_features.append(combined_feat)
            combined_labels.append(label)

        except Exception as e:
            print(f"Error in combined feature extraction for {img_path}: {e}")


In [56]:
x_train, x_test, y_train, y_test = train_test_split(
    combined_features, combined_labels, test_size=0.2, random_state=42)

forest_model.fit(x_train, y_train)
print("Combined RF Accuracy:", forest_model.score(x_test, y_test))

svm_model.fit(x_train, y_train)
print("Combined SVM Accuracy:", svm_model.score(x_test, y_test))

knn_model.fit(x_train, y_train)
print("Combined KNN Accuracy:", knn_model.score(x_test, y_test))


Combined RF Accuracy: 0.7308333333333333
Combined SVM Accuracy: 0.5504166666666667
Combined KNN Accuracy: 0.38958333333333334


# learnt features

- deep learning (resnet50) for feature extraction
- passed to traditional ml classifiers

In [57]:
import os
import numpy as np
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_deep_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)   j
    features = model.predict(x, verbose=0)
    return features.flatten()


dataset_path = path

deep_features = []
labels = []

for label in os.listdir(dataset_path):
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue

    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        try:
            features = extract_deep_features(img_path)
            deep_features.append(features)
            labels.append(label)
        except Exception as e:
            print(f"Error on {img_path}: {e}")


Exception ignored in: <function tqdm.__del__ at 0x353f971f0>
Traceback (most recent call last):
  File "/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
2025-04-19 18:27:57.545540: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-04-19 18:27:57.545567: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-04-19 18:27:57.545575: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-04-19 18:27:57.545725: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-19 18:27:57

Error on /Users/bytedance/.cache/kagglehub/datasets/ankit1743/skyview-an-aerial-landscape-dataset/versions/1/Aerial_Landscapes/Residential/.DS_Store: cannot identify image file <_io.BytesIO object at 0x53a369ea0>
Error on /Users/bytedance/.cache/kagglehub/datasets/ankit1743/skyview-an-aerial-landscape-dataset/versions/1/Aerial_Landscapes/Airport/.DS_Store: cannot identify image file <_io.BytesIO object at 0x539f92f40>
